In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote


In [2]:
driver = webdriver.Chrome('chromedriver')

In [5]:
base_url = 'https://search.naver.com/search.naver?where=view&sm=tab_jum&query='
url = base_url+quote('도깨비 펄어비스')

In [6]:
driver.get(url)

In [7]:
# blog 페이지 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for i in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

In [8]:
# html 코드 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')


In [9]:
lis = soup.select('li.bx._svp_item')
len(lis)

135

In [10]:
li = lis[0]
title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
date = li.select_one('.sub_time.sub_txt').get_text()
title, date, content

('펄어비스 도깨비 게임 플레이 분석',
 '5일 전',
 '게임 속 세상에서 다시 동심을 찾을 수 있는 요소를 가져 남녀노소 할 것 없이 게임을 즐길 수 있다는 게 장점이라 생각한다. 앞으로 펄어비스 도깨비가 잘 개발되었으면 좋겠다. 그럼, 이만.')

In [11]:
title_list, content_list, date_list = [], [],[]
for i, li in enumerate(lis):
    try:
        adv = li.select_one('a.link_ad').get_text()
        if adv.strip()=='광고':
            continue
    except:
        title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text()
        content = li.select_one('div.api_txt_lines.dsc_txt').get_text()
        date = li.select_one('.sub_time.sub_txt').get_text()

        title_list.append(title)
        content_list.append(content)
        date_list.append(date)

    
    #    print(i)


In [13]:
df = pd.DataFrame({
    'title': title_list, 'date':date_list, 'content':content_list
})

df.head()

,title,date,content
0,펄어비스 도깨비 게임 플레이 분석,5일 전,게임 속 세상에서 다시 동심을 찾을 수 있는 요소를 가져 남녀노소 할 것 없이 게임...
1,펄어비스 도깨비 총정리 주가를 떡상시킨 게임 DokeV,3일 전,지난 게임스컴 행사에서 공개된 펄어비스 신작 게임 도깨비(DokeV)의 게임 플레이...
2,[펄어비스] 도깨비 극찬! 붉은사막 대작 기대! 신작으로 기대감 업 소식,18시간 전,그것을 그대로 도깨비에 녹여 도깨비를 수집~ 다양한 재미를 얻을 수 있게~ 그리고 ...
3,펄어비스 도깨비 실제 플레이 영상(해외 유저 반응) & 주식 떡상한 게임,19시간 전,펄어비스의 도깨비 영상을 유심히 살펴보면 캐릭터의 머리카락이 하나씩 움직이거나 옷이...
4,펄어비스 신작 '도깨비' 국산 게임 이거 느낌이 상당히 좋은데?,4일 전,"마치 GTA가 포켓몬을 만난 것 같다."" - 캐나다 게임 웹진 Game Rant 펄..."


In [14]:
df.shape

(135, 3)

In [15]:
df.to_csv('data/dokev.txt')

In [16]:
driver.close()